In [2]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.2/734.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 16.9 MB/s eta 0:00:00


In [9]:
import sys
print(sys.executable)

/home/codespace/.python/current/bin/python3


In [10]:
import openai
print(openai.__version__)

1.74.0


In [7]:
!pip install ag2[openai]

In [ ]:
import autogen
import dotenv

dotenv.load_dotenv()

config_list = autogen.config_list_from_dotenv(
    ".env",
    {"gpt-3.5-turbo": "OPENAI_API_KEY"}
)

llm_config = {
    "cache_seed": 43,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,  # in seconds
}
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved "
                   "by this admin.",
    code_execution_config={
        "work_dir": "code",
        "use_docker": False
    },
    human_input_mode="TERMINATE",
)
engineer = autogen.AssistantAgent( 
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. Make sure you save code to disk.  You write python/shell
    code to solve tasks. Wrap the code in a code block that specifies the script type and the name of the file to
    save to disk.""",
)
scientist = autogen.AssistantAgent( 
    name="Scientist",
    llm_config=llm_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their
    abstracts printed. You don't write code.""",
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
""",
    llm_config=llm_config,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the "
                   "plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)
group_chat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, critic], messages=[], max_round=12
)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

user_proxy.initiate_chat(
    manager,
    message="""
Find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


Find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

```python
# arxiv_llm_papers.py

import requests
from bs4 import BeautifulSoup

# URL for arXiv search results for LLM applications in the last week
url = 'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=LLM&terms-0-field=all&classification-computer_science=y&classification-physics_archives=all&date-filter-by=last_week&size=50&order=-announced_date_first'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the papers in the search results
papers = soup.find_all('li', {'class': 'arxiv-result'})

# Create a markdown table with the domains of the papers
table = "| Paper Title | Dom

ChatResult(chat_id=None, chat_history=[{'content': '\nFind papers on LLM applications from arxiv in the last week, create a markdown table of different domains.\n', 'role': 'assistant', 'name': 'Admin'}, {'content': '```python\n# arxiv_llm_papers.py\n\nimport requests\nfrom bs4 import BeautifulSoup\n\n# URL for arXiv search results for LLM applications in the last week\nurl = \'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=LLM&terms-0-field=all&classification-computer_science=y&classification-physics_archives=all&date-filter-by=last_week&size=50&order=-announced_date_first\'\n\n# Send a GET request to the URL\nresponse = requests.get(url)\n\n# Parse the HTML content of the page\nsoup = BeautifulSoup(response.content, \'html.parser\')\n\n# Find all the papers in the search results\npapers = soup.find_all(\'li\', {\'class\': \'arxiv-result\'})\n\n# Create a markdown table with the domains of the papers\ntable = "| Paper Title | Domain |\\n| --- | --- |\\n"